In [1]:
import os
os.environ["CUDA_VISIBLE_DEVICES"] = "-1"  # Force CPU usage
os.environ["TF_CPP_MIN_LOG_LEVEL"] = "2"  # Suppress INFO and WARNING messages

import tensorflow as tf
print("TensorFlow running on:", "GPU" if tf.config.list_physical_devices('GPU') else "CPU")

TensorFlow running on: CPU


In [2]:
!pip install pyspark


In [3]:
from google.cloud import bigquery
from google.cloud import storage
import tensorflow as tf
tf.config.threading.set_inter_op_parallelism_threads(4)  # Controls operations running in parallel
tf.config.threading.set_intra_op_parallelism_threads(4)  # Controls parallelism within individual operations
from pyspark.sql import SparkSession
from pyspark.sql.functions import col, to_date
from google.cloud import bigquery
from pyspark.ml.feature import VectorAssembler, StandardScaler, OneHotEncoder, PCA
from pyspark.ml.regression import RandomForestRegressor, LinearRegression, DecisionTreeRegressor, GBTRegressor
from pyspark.ml.classification import LogisticRegression, RandomForestClassifier, DecisionTreeClassifier
from pyspark.ml.evaluation import RegressionEvaluator, MulticlassClassificationEvaluator
from pyspark.ml.feature import StringIndexer, Imputer
from pyspark.ml import Pipeline
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import warnings
warnings.filterwarnings("ignore") # Suppress all warnings
pd.options.display.float_format = '{:.2f}'.format

In [4]:
import psutil
print(f"Threads used: {psutil.Process().num_threads()}")


Threads used: 13


In [5]:
# Initialize BigQuery and Storage clients
storage_client = storage.Client()
client = bigquery.Client()
storage_client = storage.Client()
print("Authentication successful!")


Authentication successful!


In [6]:
spark = SparkSession.builder \
    .appName("BigQueryLoader") \
    .config("spark.jars", "/opt/spark/jars/spark-bigquery-latest_2.12.jar") \
    .getOrCreate()


25/02/05 07:19:48 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
----------------------------------------
Exception occurred during processing of request from ('127.0.0.1', 60242)
Traceback (most recent call last):
  File "/opt/conda/envs/tensorflow/lib/python3.10/socketserver.py", line 316, in _handle_request_noblock
    self.process_request(request, client_address)
  File "/opt/conda/envs/tensorflow/lib/python3.10/socketserver.py", line 347, in process_request
    self.finish_request(request, client_address)
  File "/opt/conda/envs/tensorflow/lib/python3.10/socketserver.py", line 360, in finish_request
    self.RequestHandlerClass(request, client_address, self)
  File "/opt/conda/envs/tensorflow/lib/python3.10/socketserver.py", line 747, in __init__
    self.handle()
  File "/opt

In [7]:
print(spark.sparkContext._jsc.hadoopConfiguration().get("spark.jars"))


None


In [8]:
from google.cloud import bigquery
from pyspark.sql import SparkSession

# Define GCP details
project_id = "big-data-engin"  # Replace with your actual GCP project ID
dataset_id = "Sample1"
table_id = "airplane2"

# Full table ID for BigQuery
full_table_id = f"{project_id}.{dataset_id}.{table_id}"

# Define GCS URI (Google Cloud Storage path)
gcs_uri = "gs://samplebucketvrs/Intelligence Analytics/og4/2019.csv"  # Replace with actual GCS path

# Initialize BigQuery client
client = bigquery.Client()

# Configure BigQuery job
job_config = bigquery.LoadJobConfig(
    source_format=bigquery.SourceFormat.CSV,
    skip_leading_rows=1,  # Skip the header row
    autodetect=True,  # Enable schema autodetection
    write_disposition=bigquery.WriteDisposition.WRITE_APPEND  # Append data if table exists
)

# Load data from GCS into BigQuery
load_job = client.load_table_from_uri(gcs_uri, full_table_id, job_config=job_config)

# Wait for job to complete before proceeding
load_job.result()
print(f"Loaded CSV file into {full_table_id} successfully.")

# Initialize Spark session (if running in a PySpark environment)
spark = SparkSession.builder \
    .appName("BigQueryLoad") \
    .getOrCreate()

# Load data from BigQuery into a PySpark DataFrame
df = spark.read \
    .format("bigquery") \
    .option("table", full_table_id) \
    .load()

# Show schema and first few rows
df.printSchema()
#df.show(5)
df.limit(5).toPandas()

Loaded CSV file into big-data-engin.Sample1.airplane2 successfully.


25/02/05 07:43:55 WARN SparkSession: Using an existing Spark session; only runtime SQL configurations will take effect.


root
 |-- FL_DATE: date (nullable = true)
 |-- OP_UNIQUE_CARRIER: string (nullable = true)
 |-- OP_CARRIER_FL_NUM: long (nullable = true)
 |-- ORIGIN: string (nullable = true)
 |-- DEST: string (nullable = true)
 |-- DEP_TIME: long (nullable = true)
 |-- DEP_DELAY: double (nullable = true)
 |-- TAXI_OUT: double (nullable = true)
 |-- WHEELS_OFF: long (nullable = true)
 |-- WHEELS_ON: long (nullable = true)
 |-- TAXI_IN: double (nullable = true)
 |-- ARR_TIME: long (nullable = true)
 |-- ARR_DELAY: double (nullable = true)
 |-- AIR_TIME: double (nullable = true)
 |-- DISTANCE: double (nullable = true)
 |-- CARRIER_DELAY: double (nullable = true)
 |-- WEATHER_DELAY: double (nullable = true)
 |-- NAS_DELAY: double (nullable = true)
 |-- SECURITY_DELAY: double (nullable = true)
 |-- LATE_AIRCRAFT_DELAY: double (nullable = true)
 |-- string_field_20: string (nullable = true)



,FL_DATE,OP_UNIQUE_CARRIER,OP_CARRIER_FL_NUM,ORIGIN,DEST,DEP_TIME,DEP_DELAY,TAXI_OUT,WHEELS_OFF,WHEELS_ON,...,ARR_TIME,ARR_DELAY,AIR_TIME,DISTANCE,CARRIER_DELAY,WEATHER_DELAY,NAS_DELAY,SECURITY_DELAY,LATE_AIRCRAFT_DELAY,string_field_20
0,2019-11-08,AS,64,PSG,WRG,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,31.00,NaN,NaN,NaN,NaN,NaN,None
1,2019-01-24,AS,64,PSG,WRG,1505.00,-15.00,2.00,1507.00,1519.00,...,1522.00,-23.00,12.00,31.00,NaN,NaN,NaN,NaN,NaN,None
2,2019-03-16,AS,64,PSG,WRG,1458.00,-27.00,9.00,1507.00,1519.00,...,1522.00,-23.00,12.00,31.00,NaN,NaN,NaN,NaN,NaN,None
3,2019-01-30,AS,65,WRG,PSG,1028.00,-17.00,4.00,1032.00,1040.00,...,1044.00,-21.00,8.00,31.00,NaN,NaN,NaN,NaN,NaN,None
4,2019-01-03,AS,64,PSG,WRG,1505.00,-24.00,11.00,1516.00,1529.00,...,1532.00,-21.00,13.00,31.00,NaN,NaN,NaN,NaN,NaN,None


In [9]:
df.describe()
columns_to_drop = [col for col in df.columns if col.lower() == 'id']

if columns_to_drop:
    df = df.drop(columns=columns_to_drop)
df = df.withColumn('FL_DATE', df['FL_DATE'].cast('timestamp'))

In [10]:
from pyspark.sql.types import BooleanType, NumericType, StringType, DateType, TimestampType
from pyspark.sql import SparkSession

def identify_column_type(df):
    column_types = []

    for field in df.schema.fields:  # Proper way to get schema fields in PySpark
        col_name = field.name
        dtype = field.dataType

        if isinstance(dtype, BooleanType):
            col_type = "Boolean"
        elif isinstance(dtype, NumericType):
            col_type = "Numerical"
        elif isinstance(dtype, StringType):
            col_type = "Categorical"
        elif isinstance(dtype, (DateType, TimestampType)):
            col_type = "Datetime"
        else:
            col_type = "Unknown"
        
        column_types.append((col_name, col_type))

    # Convert list of tuples into a Spark DataFrame
    schema = ["Column", "Type"]
    column_types_df = spark.createDataFrame(column_types, schema=schema)
    
    return column_types_df

# Identify column types
column_types_df = identify_column_type(df)
column_types_df.show()


+-------------------+-----------+
|             Column|       Type|
+-------------------+-----------+
|            FL_DATE|   Datetime|
|  OP_UNIQUE_CARRIER|Categorical|
|  OP_CARRIER_FL_NUM|  Numerical|
|             ORIGIN|Categorical|
|               DEST|Categorical|
|           DEP_TIME|  Numerical|
|          DEP_DELAY|  Numerical|
|           TAXI_OUT|  Numerical|
|         WHEELS_OFF|  Numerical|
|          WHEELS_ON|  Numerical|
|            TAXI_IN|  Numerical|
|           ARR_TIME|  Numerical|
|          ARR_DELAY|  Numerical|
|           AIR_TIME|  Numerical|
|           DISTANCE|  Numerical|
|      CARRIER_DELAY|  Numerical|
|      WEATHER_DELAY|  Numerical|
|          NAS_DELAY|  Numerical|
|     SECURITY_DELAY|  Numerical|
|LATE_AIRCRAFT_DELAY|  Numerical|
+-------------------+-----------+
only showing top 20 rows



In [11]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import col, mean, stddev, when, count, expr
from pyspark.sql.window import Window

# Initialize Spark Session
spark = SparkSession.builder.appName("OutlierDetection").getOrCreate()

def detect_outliers_large(df, method="iqr", threshold=1.5):
    """
    Detect outliers in a PySpark DataFrame using IQR or Z-score.
    Returns a dictionary of outlier counts per column.
    """
    outliers = {}

    if method == "iqr":
        for col_name in df.columns:
            if df.select(col_name).schema.fields[0].dataType.simpleString() in ["int", "double", "float"]:
                # Compute Q1, Q3, and IQR using Spark SQL functions
                Q1 = df.approxQuantile(col_name, [0.25], 0.01)[0]
                Q3 = df.approxQuantile(col_name, [0.75], 0.01)[0]
                IQR = Q3 - Q1
                lower_bound = Q1 - threshold * IQR
                upper_bound = Q3 + threshold * IQR

                # Count outliers
                outlier_count = df.filter((col(col_name) < lower_bound) | (col(col_name) > upper_bound)).count()
                outliers[col_name] = outlier_count

    elif method == "zscore":
        for col_name in df.columns:
            if df.select(col_name).schema.fields[0].dataType.simpleString() in ["int", "double", "float"]:
                stats = df.select(
                    mean(col(col_name)).alias("mean"),
                    stddev(col(col_name)).alias("stddev")
                ).collect()[0]

                mean_val, std_val = stats["mean"], stats["stddev"]
                
                # Count outliers
                outlier_count = df.filter(((col(col_name) - mean_val) / std_val).abs() > threshold).count()
                outliers[col_name] = outlier_count

    else:
        raise ValueError("Method must be 'iqr' or 'zscore'.")
    
    return outliers

def calculate_outlier_percentage(outliers_dict, total_rows):
    """
    Calculate the percentage of outliers for each column.
    """
    outlier_percentage = {col: (count / total_rows) * 100 for col, count in outliers_dict.items()}
    return outlier_percentage

def clean_or_winsorize(df, outliers_dict, threshold=5):
    """
    Clean or apply Winsorization based on outlier percentage.

    Parameters:
        df (Spark DataFrame): Input DataFrame.
        outliers_dict (dict): Dictionary with outlier counts.
        threshold (int): Percentage threshold to decide action.
    
    Returns:
        Spark DataFrame: Processed DataFrame.
    """
    total_rows = df.count()
    outlier_percentage = calculate_outlier_percentage(outliers_dict, total_rows)

    for col_name, percentage in outlier_percentage.items():
        if percentage <= threshold:
            # Remove outliers
            Q1 = df.approxQuantile(col_name, [0.25], 0.01)[0]
            Q3 = df.approxQuantile(col_name, [0.75], 0.01)[0]
            IQR = Q3 - Q1
            lower_bound = Q1 - 1.5 * IQR
            upper_bound = Q3 + 1.5 * IQR

            df = df.filter((col(col_name) >= lower_bound) & (col(col_name) <= upper_bound))
            print(f"Removed {percentage:.2f}% outliers from column '{col_name}'.")
        else:
            # Apply Winsorization using percentile approximation
            lower_bound = df.approxQuantile(col_name, [0.05], 0.01)[0]
            upper_bound = df.approxQuantile(col_name, [0.95], 0.01)[0]

            df = df.withColumn(
                col_name,
                when(col(col_name) < lower_bound, lower_bound)
                .when(col(col_name) > upper_bound, upper_bound)
                .otherwise(col(col_name))
            )
            print(f"Applied Winsorization to column '{col_name}' with {percentage:.2f}% outliers.")

    return df

# Example Usage:
# Load dataset from BigQuery or CSV
# df = spark.read.format("bigquery").option("table", "your_project.your_dataset.your_table").load()
# df = spark.read.csv("your_large_dataset.csv", header=True, inferSchema=True)

total_rows = df.count()
outliers_dict = detect_outliers_large(df, method="iqr", threshold=1.5)

# Process data based on outlier percentage
df_processed = clean_or_winsorize(df, outliers_dict, threshold=5)

# Save the processed dataset
# df_processed.write.csv("cleaned_dataset.csv", header=True)


25/02/05 07:44:15 WARN SparkSession: Using an existing Spark session; only runtime SQL configurations will take effect.


Applied Winsorization to column 'DEP_DELAY' with 13.41% outliers.


Removed 4.30% outliers from column 'TAXI_OUT'.


Applied Winsorization to column 'TAXI_IN' with 6.19% outliers.


Applied Winsorization to column 'ARR_DELAY' with 9.14% outliers.


...

Removed 2.16% outliers from column 'CARRIER_DELAY'.


Removed 1.11% outliers from column 'WEATHER_DELAY'.


Removed 1.46% outliers from column 'NAS_DELAY'.


Removed 0.06% outliers from column 'SECURITY_DELAY'.


Removed 1.83% outliers from column 'LATE_AIRCRAFT_DELAY'.


In [12]:
"""from pyspark.sql import SparkSession
from pyspark.sql.types import *

# Initialize Spark
spark = SparkSession.builder.appName("PandasToSpark").getOrCreate()
# Now pass it to the function
column_types_df = identify_column_type(df)
column_types_df.show()"""

'from pyspark.sql import SparkSession\nfrom pyspark.sql.types import *\n\n# Initialize Spark\nspark = SparkSession.builder.appName("PandasToSpark").getOrCreate()\n# Now pass it to the function\ncolumn_types_df = identify_column_type(df)\ncolumn_types_df.show()'

In [ ]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import col, countDistinct
from pyspark.sql.types import StructType, StructField, StringType, IntegerType
from pyspark.ml.feature import StringIndexer, OneHotEncoder, Imputer
from pyspark.ml import Pipeline

# Initialize Spark Session
spark = SparkSession.builder.appName("FeatureEngineering").getOrCreate()

# Compute unique counts for each column
unique_counts_df = (
    df.select([countDistinct(col(c)).alias(c) for c in df.columns])
    .toPandas()
    .melt(var_name="Column", value_name="Unique_Values")
)

# Convert Pandas DataFrame to PySpark DataFrame with explicit schema
unique_counts_spark_df = spark.createDataFrame(
    unique_counts_df,
    schema=StructType([
        StructField("Column", StringType(), True),
        StructField("Unique_Values", IntegerType(), True)
    ])
)

# Ensure 'column_types_df' exists and join with unique counts
if "column_types_df" in locals():
    column_info = column_types_df.join(unique_counts_spark_df, on="Column")
else:
    raise ValueError("column_types_df is not defined. Ensure you run 'identify_column_type(df)' first.")

# Initialize transformation lists
indexers, encoders, imputers = [], [], []

# Process each column's metadata
for row in column_info.rdd.collect():  
    col_name = row["Column"]
    col_type = row["Type"]
    unique_values = row["Unique_Values"]

    if col_type in ["Categorical", "Boolean"]:
        if unique_values <= 1:
            print(f"Skipping column '{col_name}' (only {unique_values} unique value).")
            continue
        
        indexer = StringIndexer(inputCol=col_name, outputCol=f"{col_name}_index", handleInvalid="keep")
        indexers.append(indexer)
        
        if unique_values <= 10:
            encoder = OneHotEncoder(inputCol=f"{col_name}_index", outputCol=f"{col_name}_encoded")
            encoders.append(encoder)

    elif col_type == "Numerical":
        imputer = Imputer(inputCols=[col_name], outputCols=[col_name + "_imputed"], strategy="median")
        imputers.append(imputer)

# Create and apply the transformation pipeline
pipeline = Pipeline(stages=indexers + encoders + imputers)
df_transformed = pipeline.fit(df).transform(df)

# Show transformed dataset
#df_transformed.show()


25/02/05 07:50:55 WARN SparkSession: Using an existing Spark session; only runtime SQL configurations will take effect.


Skipping column 'string_field_20' (only 0 unique value).


[2152.554s][warning][os,thread] Failed to start thread "Unknown thread" - pthread_create failed (EAGAIN) for attributes: stacksize: 1024k, guardsize: 0k, detached.
[2152.555s][warning][os,thread] Failed to start the native thread for java.lang.Thread "block-manager-storage-async-thread-pool-448"
[2152.555s][warning][os,thread] Failed to start thread "Unknown thread" - pthread_create failed (EAGAIN) for attributes: stacksize: 1024k, guardsize: 0k, detached.
[2152.555s][warning][os,thread] Failed to start the native thread for java.lang.Thread "block-manager-storage-async-thread-pool-449"
[2152.555s][warning][os,thread] Failed to start thread "Unknown thread" - pthread_create failed (EAGAIN) for attributes: stacksize: 1024k, guardsize: 0k, detached.
[2152.555s][warning][os,thread] Failed to start the native thread for java.lang.Thread "block-manager-storage-async-thread-pool-450"
[2152.556s][warning][os,thread] Failed to start thread "Unknown thread" - pthread_create failed (EAGAIN) for 

Exception in thread "block-manager-storage-async-thread-pool-443" java.lang.OutOfMemoryError: unable to create native thread: possibly out of memory or process/resource limits reached
	at java.base/java.lang.Thread.start0(Native Method)
	at java.base/java.lang.Thread.start(Thread.java:798)
	at java.base/java.util.concurrent.ThreadPoolExecutor.addWorker(ThreadPoolExecutor.java:937)
	at java.base/java.util.concurrent.ThreadPoolExecutor.processWorkerExit(ThreadPoolExecutor.java:1005)
	at java.base/java.util.concurrent.ThreadPoolExecutor.runWorker(ThreadPoolExecutor.java:1142)
	at java.base/java.util.concurrent.ThreadPoolExecutor$Worker.run(ThreadPoolExecutor.java:628)
	at java.base/java.lang.Thread.run(Thread.java:829)
java.lang.IllegalStateException: problem in scala.concurrent internal callback
	at scala.concurrent.Future$InternalCallbackExecutor$.reportFailure(Future.scala:877)
	at scala.concurrent.impl.CallbackRunnable.executeWithValue(Promise.scala:72)
	at scala.concurrent.impl.Promi

[2152.999s][warning][os,thread] Failed to start thread "Unknown thread" - pthread_create failed (EAGAIN) for attributes: stacksize: 1024k, guardsize: 0k, detached.
[2152.999s][warning][os,thread] Failed to start the native thread for java.lang.Thread "Gax-9246"
[2153.002s][warning][os,thread] Failed to start thread "Unknown thread" - pthread_create failed (EAGAIN) for attributes: stacksize: 1024k, guardsize: 0k, detached.
[2153.002s][warning][os,thread] Failed to start the native thread for java.lang.Thread "Gax-9247"
[2153.004s][warning][os,thread] Failed to start thread "Unknown thread" - pthread_create failed (EAGAIN) for attributes: stacksize: 1024k, guardsize: 0k, detached.
[2153.004s][warning][os,thread] Failed to start the native thread for java.lang.Thread "Gax-9248"
[2153.006s][warning][os,thread] Failed to start thread "Unknown thread" - pthread_create failed (EAGAIN) for attributes: stacksize: 1024k, guardsize: 0k, detached.
[2153.006s][warning][os,thread] Failed to start th

25/02/05 07:55:39 ERROR Executor: Exception in task 2.0 in stage 175.0 (TID 1571)
java.lang.OutOfMemoryError: unable to create native thread: possibly out of memory or process/resource limits reached
	at java.base/java.lang.Thread.start0(Native Method)
	at java.base/java.lang.Thread.start(Thread.java:798)
	at java.base/java.util.concurrent.ThreadPoolExecutor.addWorker(ThreadPoolExecutor.java:937)
	at java.base/java.util.concurrent.ThreadPoolExecutor.ensurePrestart(ThreadPoolExecutor.java:1583)
	at java.base/java.util.concurrent.ScheduledThreadPoolExecutor.delayedExecute(ScheduledThreadPoolExecutor.java:346)
	at java.base/java.util.concurrent.ScheduledThreadPoolExecutor.scheduleAtFixedRate(ScheduledThreadPoolExecutor.java:632)
	at com.google.cloud.spark.bigquery.repackaged.com.google.api.gax.rpc.Watchdog.start(Watchdog.java:94)
	at com.google.cloud.spark.bigquery.repackaged.com.google.api.gax.rpc.Watchdog.create(Watchdog.java:82)
	at com.google.cloud.spark.bigquery.repackaged.com.google

Py4JError: An error occurred while calling o951.fit

In [ ]:
df_transformed.limit(5).toPandas()

In [ ]:
from pyspark.sql import SparkSession
from pyspark.ml.feature import VectorAssembler, StandardScaler
from pyspark.ml.regression import RandomForestRegressor
from pyspark.ml.classification import RandomForestClassifier
from pyspark.ml import Pipeline
from pyspark.ml.evaluation import RegressionEvaluator, BinaryClassificationEvaluator
from pyspark.sql.functions import col

# Initialize Spark session
spark = SparkSession.builder.appName("Modeling").getOrCreate()


# Sample 90% of the data
sampled_df = df_transformed.sample(fraction=0.9, seed=42)

# Separate features (X) and target (y)
y = "DEP_DELAY"
X = sampled_df.drop(*["ARR_DELAY", "FL_DATE"])

# Convert numerical columns to features
numeric_cols = [c for c in X.columns if dict(X.dtypes)[c] in ['int', 'double']]

# Create a feature vector by combining all columns into a single vector column
assembler = VectorAssembler(inputCols=numeric_cols, outputCol="features")
X_assembled = assembler.transform(X)

# Scale only numerical columns
scaler = StandardScaler(inputCol="features", outputCol="scaled_features")
scaler_model = scaler.fit(X_assembled)
X_scaled = scaler_model.transform(X_assembled)

# Train/Test Split (80/20)
train_df, test_df = X_scaled.randomSplit([0.8, 0.2], seed=42)

# Define the model based on the target variable type (Regression or Classification)
if dict(sampled_df.dtypes)[y] in ['int', 'double']:
    # Regression Task
    model = RandomForestRegressor(featuresCol="scaled_features", labelCol=y, numTrees=100, seed=42)
    evaluator = RegressionEvaluator(labelCol=y)
    task_type = "Regression"
else:
    # Classification Task
    model = RandomForestClassifier(featuresCol="scaled_features", labelCol=y, numTrees=100, seed=42)
    evaluator = BinaryClassificationEvaluator(labelCol=y)
    task_type = "Classification"

# Train the model
model_fitted = model.fit(train_df)

# Get Feature Importances (Not directly available in PySpark, so we get model's feature importances)
feature_importances = model_fitted.featureImportances

# Convert the importances into a DataFrame for easy reading
importance_df = pd.DataFrame({
    'Feature': numeric_cols,
    'Importance': feature_importances.toArray()
})

# Show top features
importance_df.head()
